In [1]:
# Step3A.1
import pandas as pd
import numpy as np
from itertools import product

# Load your existing monthly SKU data
monthly_data = pd.read_csv("all_sku_monthly.csv")
# Convert to end of month to align with Prophet forecast output
monthly_data['YearMonth'] = pd.to_datetime(monthly_data['YearMonth'].astype(str)) + pd.offsets.MonthEnd(0)




In [2]:
# Step3A.2 Create Complete SKU-Month Combinations (Including Zero Sales Months)

# Define the full date range
full_date_range = pd.date_range(start='2023-01-01', end='2025-03-31', freq='ME')

# Get all unique SKUs
sku_list = monthly_data['Item_ID'].unique()

# Create all combinations of SKUs and dates
sku_month_combinations = pd.DataFrame(list(product(sku_list, full_date_range)), columns=['Item_ID', 'YearMonth'])

# Merge with actual data
all_sku_monthly_w0 = pd.merge(sku_month_combinations, monthly_data, on=['Item_ID', 'YearMonth'], how='left')

# Fill missing Monthly_Quantity with 0
all_sku_monthly_w0['Monthly_Quantity'] = all_sku_monthly_w0['Monthly_Quantity'].fillna(0)

# Save for evaluation
all_sku_monthly_w0.to_csv("all_sku_monthly_w0.csv", index=False)


# Preview
all_sku_monthly_w0.head()


,Item_ID,YearMonth,Monthly_Quantity
0,610009,2023-01-31,6.0
1,610009,2023-02-28,4.0
2,610009,2023-03-31,4.0
3,610009,2023-04-30,10.0
4,610009,2023-05-31,6.0


In [3]:
# Step3A.3 Set Training and Test Set

# Training set (historical)
training_data = all_sku_monthly_w0[
    (all_sku_monthly_w0['YearMonth'] >= '2023-01-31') & 
    (all_sku_monthly_w0['YearMonth'] <= '2024-08-31')
]

# Test set (evaluation)
test_data = all_sku_monthly_w0[
    (all_sku_monthly_w0['YearMonth'] >= '2024-09-30') & 
    (all_sku_monthly_w0['YearMonth'] <= '2025-03-31')
]


In [4]:
# Step3A.4 Forecast Using Prophet

from prophet import Prophet
from tqdm import tqdm
import warnings


#Imports Python's built-in warnings module and suppresses warning messages. This cleans up the output by hiding minor warnings (common with Prophet).
warnings.filterwarnings("ignore")

# Store forecasts
prophet_forecasts_nofilter = []

# Forecast loop
for sku in tqdm(sku_list):
    sku_train_df = training_data[training_data['Item_ID'] == sku][['YearMonth', 'Monthly_Quantity']]

    # Skip SKUs with zero historical sales
    if sku_train_df['Monthly_Quantity'].sum() == 0:
        continue

    # Prepare data for Prophet
    prophet_df = sku_train_df.rename(columns={'YearMonth': 'ds', 'Monthly_Quantity': 'y'})

    try:
        model = Prophet()
        model.fit(prophet_df)

        future = model.make_future_dataframe(periods=7, freq='M')
        forecast = model.predict(future)

        forecast['Item_ID'] = sku
        prophet_forecasts_nofilter.append(forecast)

    except Exception as e:
        print(f"SKU {sku} error: {e}")

# Combine results
prophet_forecast_df = pd.concat(prophet_forecasts_nofilter, ignore_index=True)
prophet_forecast_df.rename(columns={'ds': 'ForecastMonth', 'yhat': 'Forecasted_Quantity'}, inplace=True)

# ✅ Clean Prophet forecasts (no negative or near-zero values)
prophet_forecast_df['Forecasted_Quantity'] = prophet_forecast_df['Forecasted_Quantity'].apply(
    lambda x: 0 if x < 0 or abs(x) < 1e-5 else round(x, 2)
)

# Save forecast data
prophet_forecast_df.to_csv("prophet_full_forecast.csv", index=False)


  0%|          | 0/2935 [00:00<?, ?it/s]16:35:43 - cmdstanpy - INFO - Chain [1] start processing
16:35:43 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 1/2935 [00:00<17:42,  2.76it/s]16:35:44 - cmdstanpy - INFO - Chain [1] start processing
16:35:44 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 2/2935 [00:00<12:50,  3.81it/s]16:35:44 - cmdstanpy - INFO - Chain [1] start processing
16:35:44 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 3/2935 [00:00<10:58,  4.45it/s]16:35:44 - cmdstanpy - INFO - Chain [1] start processing
16:35:44 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 4/2935 [00:00<09:59,  4.89it/s]16:35:44 - cmdstanpy - INFO - Chain [1] start processing
16:35:44 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 5/2935 [00:01<09:17,  5.25it/s]16:35:44 - cmdstanpy - INFO - Chain [1] start processing
16:35:44 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 6/2935 [00:01<08:53,  5.4

In [5]:
# Step3A.5 Compare Forecast with Actuals

# Prepare DataFrames
actual_df = all_sku_monthly_w0.copy()
actual_df.rename(columns={'YearMonth': 'ForecastMonth', 'Monthly_Quantity': 'Actual'}, inplace=True)

forecast_df = prophet_forecast_df.copy()

# Merge Actuals LEFT JOIN Forecasts
comparison_df = pd.merge(
    actual_df[['ForecastMonth', 'Item_ID', 'Actual']],
    forecast_df[['ForecastMonth', 'Item_ID', 'Forecasted_Quantity']],
    on=['ForecastMonth', 'Item_ID'],
    how='left'  # Keep all actuals
)

# Fill missing Forecasted_Quantity with 0
comparison_df['Forecasted_Quantity'] = comparison_df['Forecasted_Quantity'].fillna(0)

# (Optional) Calculate Forecast_Error
comparison_df['Forecast_Error'] = comparison_df['Forecasted_Quantity'] - comparison_df['Actual']

# Save the full comparison
comparison_df.to_csv("actual_vs_prophet_forecasts.csv", index=False)

print("✅ Full Forecast vs Actual (Full Lineup) Saved: 'actual_vs_prophet_forecasts.csv'")


✅ Full Forecast vs Actual (Full Lineup) Saved: 'actual_vs_prophet_forecasts.csv'


In [6]:
# Step3A.6 - Evaluate Prophet Forecast Accuracy

import pandas as pd
from sklearn.metrics import (
    root_mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error
)

# Load and prepare forecast comparison file
comparison_df = pd.read_csv("actual_vs_prophet_forecasts.csv")
comparison_df['ForecastMonth'] = pd.to_datetime(comparison_df['ForecastMonth'])

# Filter to forecast horizon only
eval_df = comparison_df[comparison_df['ForecastMonth'] >= '2024-09-30']

# Compute metrics
rmse = root_mean_squared_error(eval_df['Actual'], eval_df['Forecasted_Quantity'])
mae = mean_absolute_error(eval_df['Actual'], eval_df['Forecasted_Quantity'])

# Avoid divide-by-zero issue in MAPE by filtering out zero-actual rows
nonzero_eval_df = eval_df[eval_df['Actual'] != 0]
mape = mean_absolute_percentage_error(
    nonzero_eval_df['Actual'],
    nonzero_eval_df['Forecasted_Quantity']
) * 100

# Display results
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")


RMSE: 16.20
MAE: 6.27
MAPE: 109.72%
